In [1]:
import os
import gc
import time
import pickle
import pandas as pd
import numpy as np
from contextlib import contextmanager

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [3]:
def reduce_mem_usage(df, verbose=False):

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Memory usage: Before={:.2f} MB, After={:.2f} MB, Decreased by={:.1f}%'\
                  .format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
def one_hot_encoder(df, categorical_columns):
    original_columns = list(df.columns)
    df = pd.get_dummies(df, columns = categorical_columns, dummy_na = True)
    cat_columns = [c for c in df.columns if c not in original_columns]
    cont_columns = [c for c in df.columns if c in original_columns]
    return df, cat_columns, cont_columns

In [18]:
# Application_train/test.csv
def application_train_test(sample_size = None):
    app_train_df = reduce_mem_usage(pd.read_csv('./input/application_train.csv', nrows = sample_size))
    len_train = len(app_train_df)
    app_test_df = reduce_mem_usage(pd.read_csv('./input/application_test.csv', nrows = sample_size))
    app_data = pd.concat([app_train_df, app_test_df], sort=True)
    #df = df[df['CODE_GENDER'] != 'XNA']
    
    app_data_cat = app_data.columns[app_data.dtypes == 'object'].tolist() + ['HOUR_APPR_PROCESS_START']
    app_data['HOUR_APPR_PROCESS_START'] = app_data['HOUR_APPR_PROCESS_START'] // 3
    app_data[app_data_cat] = app_data[app_data_cat].astype('object')

    app_data['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    app_data['NEW_CREDIT_TO_ANNUITY'] = app_data['AMT_CREDIT'] / app_data['AMT_ANNUITY']
    app_data['NEW_ANN_LENGTH_EMPLOYED_RATIO'] = app_data['NEW_CREDIT_TO_ANNUITY'] / app_data['DAYS_EMPLOYED']
    app_data['NEW_ANNUITY_INCOME_RATIO'] = app_data['AMT_ANNUITY'] / (1 + app_data['AMT_INCOME_TOTAL'])
    app_data['NEW_CREDIT_TO_INCOME_RATIO'] = app_data['AMT_CREDIT'] / app_data['AMT_INCOME_TOTAL']
    app_data['NEW_INCOME_PER_PERSON'] = app_data['AMT_INCOME_TOTAL'] / app_data['CNT_FAM_MEMBERS']
    app_data['NEW_CREDIT_TO_GOODS_RATIO'] = app_data['AMT_CREDIT'] / app_data['AMT_GOODS_PRICE']
    app_data['NEW_CHILDREN_RATIO'] = app_data['CNT_CHILDREN'] / app_data['CNT_FAM_MEMBERS']
    app_data['NEW_WORKING_LIFE_RATIO'] = app_data['DAYS_EMPLOYED'] / app_data['DAYS_BIRTH']
    app_data['NEW_LOAN_INCOME_RATIO'] = app_data['AMT_CREDIT'] / app_data['AMT_INCOME_TOTAL']
    app_data['NEW_TOTAL_DOCS_SUBMITTED'] = app_data.loc[:, app_data.columns.str.contains('FLAG_DOCUMENT')].sum(axis=1)
    
    app_data['NEW_SOURCES_PROD'] = app_data['EXT_SOURCE_1'] * app_data['EXT_SOURCE_2'] * app_data['EXT_SOURCE_3']
    app_data['NEW_EXT_SOURCES_MEAN'] = app_data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)    
    app_data['NEW_SCORES_STD'] = app_data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    app_data['NEW_SCORES_STD'] = app_data['NEW_SCORES_STD'].fillna(app_data['NEW_SCORES_STD'].mean())
    app_data['NEW_PHONE_TO_BIRTH_RATIO'] = app_data['DAYS_LAST_PHONE_CHANGE'] / app_data['DAYS_BIRTH']
    app_data['NEW_CREDIT_TO_INCOME_RATIO'] = app_data['AMT_CREDIT'] / app_data['AMT_INCOME_TOTAL']
     
    docs = [_f for _f in app_data.columns if 'FLAG_DOC' in _f]
    live = [_f for _f in app_data.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
    inc_by_org = app_data[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
    
    app_data['NEW_INC_BY_ORG'] = app_data['ORGANIZATION_TYPE'].map(inc_by_org)
    app_data['NEW_INC_PER_CHLD'] = app_data['AMT_INCOME_TOTAL'] / (1 + app_data['CNT_CHILDREN'])
    app_data['NEW_DOC_IND_KURT'] = app_data[docs].kurtosis(axis=1)
    app_data['NEW_LIVE_IND_SUM'] = app_data[live].sum(axis=1)
    app_data['NEW_CAR_TO_BIRTH_RATIO'] = app_data['OWN_CAR_AGE'] / app_data['DAYS_BIRTH']
    app_data['NEW_CAR_TO_EMPLOY_RATIO'] = app_data['OWN_CAR_AGE'] / app_data['DAYS_EMPLOYED']
    app_data['NEW_PHONE_TO_BIRTH_RATIO'] = app_data['DAYS_LAST_PHONE_CHANGE'] / app_data['DAYS_EMPLOYED']
    
    app_data, app_data_cat, app_data_cont = one_hot_encoder(app_data, app_data_cat)

    return app_data, len_train

In [6]:
def bureau_add(bureau_df):
    
    def f1(x):
        if x == 'Closed':
            y = 0
        else:
            y = 1    
        return y

    def f2(x):  
        if x<0: 
            y = 0 
        else: 
            y = 1 
        return y 
    
    b_copy = bureau_df.copy(deep=True)
    
        # Number of Loans per Customer
    bureau_add = b_copy[['SK_ID_CURR', 'DAYS_CREDIT']].groupby('SK_ID_CURR', )['DAYS_CREDIT']\
        .count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
    
        # Number of types of Credit loans for each Customer 
    temp = b_copy[['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE']\
        .nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
    bureau_add = bureau_add.merge(temp, on = ['SK_ID_CURR'], how = 'left')
    
        # Average Number of Loans per Loan Type
    bureau_add['AVERAGE_LOAN_TYPE'] = bureau_add['BUREAU_LOAN_COUNT']/bureau_add['BUREAU_LOAN_TYPES']
    # Create a new dummy column for whether CREDIT is ACTIVE OR CLOSED
    b_copy['CREDIT_ACTIVE_BINARY'] = b_copy.apply(lambda x: f1(x.CREDIT_ACTIVE), axis = 1)
    # Calculate mean number of loans that are ACTIVE per CUSTOMER 
    temp = b_copy.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY']\
        .mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
    bureau_add = bureau_add.merge(temp, on = ['SK_ID_CURR'], how = 'left')
    
        # The Ratio of Total Debt to Total Credit for each Customer
    b_copy['AMT_CREDIT_SUM_DEBT'] = b_copy['AMT_CREDIT_SUM_DEBT'].fillna(0)
    b_copy['AMT_CREDIT_SUM'] = b_copy['AMT_CREDIT_SUM'].fillna(0)
    temp = b_copy[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT']\
        .sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
    bureau_add = bureau_add.merge(temp, on = ['SK_ID_CURR'], how = 'left')
    temp = b_copy[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM']\
        .sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})
    bureau_add = bureau_add.merge(temp, on = ['SK_ID_CURR'], how = 'left')
    bureau_add['DEBT_CREDIT_RATIO'] = bureau_add['TOTAL_CUSTOMER_DEBT']/bureau_add['TOTAL_CUSTOMER_CREDIT']
    bureau_add = bureau_add.replace([np.inf, -np.inf], np.nan)
    del bureau_add['TOTAL_CUSTOMER_DEBT'], bureau_add['TOTAL_CUSTOMER_CREDIT']

        # Average Number of Loans Prolonged
    b_copy['CNT_CREDIT_PROLONG'] = b_copy['CNT_CREDIT_PROLONG'].fillna(0)
    temp = b_copy[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']].groupby(by = ['SK_ID_CURR'])['CNT_CREDIT_PROLONG']\
        .mean().reset_index().rename( index = str, columns = { 'CNT_CREDIT_PROLONG': 'AVG_CREDITDAYS_PROLONGED'})
    bureau_add = bureau_add.merge(temp, on = ['SK_ID_CURR'], how = 'left')
    
        # Average end Date
    # Dummy column to calculate 1 or 0 values. 1 for Positive CREDIT_ENDDATE and 0 for Negative
    b_copy['CREDIT_ENDDATE_BINARY'] = b_copy['DAYS_CREDIT_ENDDATE']
    b_copy['CREDIT_ENDDATE_BINARY'] = b_copy.apply(lambda x: f2(x.DAYS_CREDIT_ENDDATE), axis = 1)
    temp = b_copy[b_copy['CREDIT_ENDDATE_BINARY'] == 1]
    #Calculate Difference in successive future end dates of CREDIT 
    # Create Dummy Column for CREDIT_ENDDATE 
    temp['DAYS_CREDIT_ENDDATE1'] = temp['DAYS_CREDIT_ENDDATE']
    temp = temp[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT_ENDDATE1']].groupby(by = ['SK_ID_CURR'])
    # Sort the values of CREDIT_ENDDATE for each customer ID 
    temp = temp.apply(lambda x: x.sort_values(['DAYS_CREDIT_ENDDATE1'], ascending = True)).reset_index(drop = True)
    # Calculate the Difference in ENDDATES and fill missing values with zero 
    temp['DAYS_ENDDATE_DIFF'] = temp.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT_ENDDATE1'].diff()
    temp['DAYS_ENDDATE_DIFF'] = temp['DAYS_ENDDATE_DIFF'].fillna(0).astype('uint32')
    del temp['DAYS_CREDIT_ENDDATE1'], temp['SK_ID_BUREAU']
    gc.collect()
    b_copy = b_copy.merge(temp, on = ['SK_ID_CURR'], how = 'left')
    # Calculate Average of DAYS_ENDDATE_DIFF
    temp = b_copy[['SK_ID_CURR', 'DAYS_ENDDATE_DIFF']].groupby(by = ['SK_ID_CURR'])['DAYS_ENDDATE_DIFF']\
        .mean().reset_index().rename( index = str, columns = {'DAYS_ENDDATE_DIFF': 'AVG_ENDDATE_FUTURE'})
    bureau_add = bureau_add.merge(temp, on = ['SK_ID_CURR'], how = 'left')
    bureau_add.set_index('SK_ID_CURR', inplace=True)
    bureau_add.columns = pd.Index(['BURO_ADD_' + e for e in bureau_add.columns.tolist()])
    
    return bureau_add

In [7]:
# bureau.csv, bureau_balance.csv
def bureau_and_balance(sample_size = None):
    bureau = reduce_mem_usage(pd.read_csv('./input/bureau.csv', nrows = sample_size))
    bb = reduce_mem_usage(pd.read_csv('./input/bureau_balance.csv', nrows = sample_size))

    bureau_additional = bureau_add(bureau)
    
    most_recent_index = bb.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].idxmax()
    bb_last = bb.iloc[most_recent_index,:]
    bb_last.drop(['MONTHS_BALANCE'], inplace=True, axis=1)
    bb_last.columns = ['SK_ID_BUREAU', 'STATUS_LAST']
    bb = bb.merge(bb_last, how='left', on='SK_ID_BUREAU')

    bb, bb_cat, bb_cont = one_hot_encoder(bb, bb.columns[bb.dtypes == 'object'].tolist())
    bureau, bureau_cat, bureau_cont = one_hot_encoder(bureau, bureau.columns[bureau.dtypes == 'object'].tolist())

    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['mean', 'var'], # 'min', 'max'
        'DAYS_CREDIT_UPDATE': ['mean'], # 'min', 'max'
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    
    # categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACT_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    bureau_agg = bureau_agg.join(bureau_additional, how='left', on='SK_ID_CURR')
    
    del active, active_agg, bureau, bureau_additional, bb_last
    gc.collect()

    return bureau_agg

In [8]:
### Previous_applications.csv
def previous_applications(sample_size = None):
    prev_app_df = pd.read_csv('./input/previous_application.csv', nrows = sample_size)

    prev_app_cat = prev_app_df.columns[prev_app_df.dtypes == 'object'].tolist() + ['HOUR_APPR_PROCESS_START']
    prev_app_df['HOUR_APPR_PROCESS_START'] = prev_app_df['HOUR_APPR_PROCESS_START'] // 3
    prev_app_df[prev_app_cat] = prev_app_df[prev_app_cat].astype('object')
    prev_app_df, prev_app_cat, prev_app_cont = one_hot_encoder(prev_app_df, prev_app_cat)

    prev_app_df['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev_app_df['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev_app_df['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev_app_df['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev_app_df['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
    prev_app_df['APP_CREDIT_PERC'] = prev_app_df['AMT_APPLICATION'] / prev_app_df['AMT_CREDIT']

    agg_funs = {'SK_ID_CURR': 'count'}
    prev_apps = prev_app_df.groupby('SK_ID_CURR').agg(agg_funs)
    prev_apps.columns = ['PREV_APP_COUNT']

    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'sum'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['max', 'mean'],
        'APP_CREDIT_PERC': ['max', 'mean', 'var'], 
        'CNT_PAYMENT': ['mean', 'sum'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'DAYS_FIRST_DRAWING': ['max', 'mean'],
        'DAYS_FIRST_DUE': ['max', 'mean'],
        'DAYS_LAST_DUE_1ST_VERSION': ['max', 'mean'],
        'DAYS_LAST_DUE': ['max', 'mean'],
        'DAYS_TERMINATION': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['max', 'mean'],
        'NFLAG_LAST_APPL_IN_DAY': ['mean'],
        'NFLAG_INSURED_ON_APPROVAL': ['mean']
    }

    cat_aggregations = {}
    for cat in prev_app_cat:
        cat_aggregations[cat] = ['mean']

    prev_app_agg = prev_app_df.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_app_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_app_agg.columns.tolist()])

    # Previous Applications: Approved Applications - only numerical features
    approved = prev_app_df[prev_app_df['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APR_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_app_agg = prev_app_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # Previous Applications: Refused Applications - only numerical features
    refused = prev_app_df[prev_app_df['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REF_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_app_agg = prev_app_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    
    prev_app_agg = prev_app_agg.join(prev_apps, how='left', on='SK_ID_CURR')    
    del refused, refused_agg, approved, approved_agg, prev_app_df, prev_apps
    gc.collect()

    return prev_app_agg

In [9]:
# POS_CASH_balance.csv
def pos_cash(sample_size = None):
    pos = reduce_mem_usage(pd.read_csv('./input/POS_CASH_balance.csv', nrows = sample_size))
    pos, pos_cat, pos_cont = one_hot_encoder(pos, pos.columns[pos.dtypes == 'object'].tolist())
    
    # Weighted average
    wm = lambda x: np.average(x, weights=-1/pos.loc[x.index, 'MONTHS_BALANCE'])
    
    aggregations = {
        'CNT_INSTALMENT': wm, 
        'CNT_INSTALMENT_FUTURE': wm, 
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean', wm],
        'SK_DPD_DEF': ['max', 'mean', wm]
    }
    for cat in pos_cat:
        aggregations[cat] = ['mean']
        
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    del pos
    gc.collect()
    
    return pos_agg

In [10]:
# installments_payments.csv
def installments_payments(sample_size = None):
    ins = reduce_mem_usage(pd.read_csv('./input/installments_payments.csv', nrows = sample_size))
    ins, ins_cat, ins_cont  = one_hot_encoder(ins, ins.columns[ins.dtypes == 'object'].tolist())
    
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Numeric features
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in ins_cat:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INS_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Count installments accounts
    ins_agg['INS_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()

    return ins_agg

In [11]:
# Credit_card_balance.csv
def credit_card_balance(sample_size = None):
    credit_card = pd.read_csv('./input/credit_card_balance.csv', nrows = sample_size)
    credit_card_cat = credit_card.columns[credit_card.dtypes == 'object'].tolist()
    credit_card, credit_card_cat, credit_card_cont = one_hot_encoder(credit_card, credit_card_cat)

    credit_card.drop(columns = ['SK_ID_PREV'], inplace = True)
    weights = lambda x: np.average(x, weights=-1/credit_card.loc[x.index, 'MONTHS_BALANCE'])
    credit_card_agg = credit_card.groupby('SK_ID_CURR').agg(['max', 'mean', 'sum', 'var', weights])
    credit_card_agg.columns = pd.Index(['CARD_' + e[0] + "_" + e[1].upper() for e in credit_card_agg.columns.tolist()])
    credit_card_agg['CARD_COUNT'] = credit_card.groupby('SK_ID_CURR').size()
    del credit_card
    gc.collect()

    return credit_card_agg

In [12]:
def merge(sample_size = None):
    df, len_train = application_train_test(sample_size)
    print(len_train)
    print("Applications df shape:", df.shape)
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(sample_size)
        print("Bureau df shape:", bureau.shape)
        df = df.merge(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(sample_size)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(sample_size)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(sample_size)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(sample_size)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    return df, len_train

In [20]:
#merged_df = merge(sample_size = 5000)
merged_df = merge()
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)

# separate metadata
meta_cols = ['SK_ID_CURR']
meta_df = merged_df[meta_cols]
merged_df.drop(meta_cols, axis=1, inplace=True)

merged_df = reduce_mem_usage(merged_df, verbose=True)

C:\Users\1\Anaconda3\lib\site-packages\pandas\core\nanops.py:121: RuntimeWarning: Mean of empty slice
  result = bn_func(values, axis=axis, **kwds)


Applications df shape: (356255, 290)


C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\1\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Bureau df shape: (305811, 109)
Process bureau and bureau_balance - done in 377s
Previous applications df shape: (338857, 292)
Process previous_applications - done in 43s
Pos-cash balance df shape: (337252, 22)
Process POS-CASH balance - done in 728s
Installments payments df shape: (339587, 26)
Process installments payments - done in 46s
Credit card balance df shape: (103558, 169)
Process credit card balance - done in 1533s


C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less
C:\Users\1\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in less


Memory usage: Before=2241.00 MB, After=690.37 MB, Decreased by=69.2%


In [21]:
merged_df.to_pickle('./data/merged_df.pkl')
meta_df.to_pickle('./data/meta_df.pkl')

### Temp